<a href="https://colab.research.google.com/github/SamuelHortadeFaria/IA_ipynb-Lista7/blob/main/Lista7_KMeans_SamuelHorta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resolução da Lista 7 - Agrupamento (KMeans com Iris)
Aluno: Samuel Horta de Faria - 801528

Disciplina: Inteligência Artificial  
Professora: Cristiane Neri Nobre

In [ ]:
!pip install plotly --upgrade
!pip install kneed
!pip install minisom

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from sklearn.decomposition import PCA
from kneed import KneeLocator
from minisom import MiniSom

## Questão 1 - Pré-processamento

In [ ]:
base = pd.read_csv('/content/sample_data/iris.csv', encoding='cp1252')
Entrada = base.iloc[:, 0:4].values

scaler = MinMaxScaler()
Entrada = scaler.fit_transform(Entrada)

from scipy.stats import zscore
z_scores = np.abs(zscore(Entrada))
Entrada = Entrada[(z_scores < 3).all(axis=1)]

Entrada.shape

## Questão 2 - Avaliação dos Agrupamentos

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(Entrada)
    wcss.append(kmeans.inertia_)

kl = KneeLocator(range(1, 11), wcss, curve="convex", direction="decreasing")
print("Número ideal de clusters (Elbow):", kl.elbow)

sil = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(Entrada)
    sil.append(silhouette_score(Entrada, kmeans.labels_))

plt.plot(range(2,11), sil)
plt.xlabel("k")
plt.ylabel("Silhouette Score")
plt.show()

## Questão 3 - Hiperparâmetros

In [ ]:
modelo = KMeans(n_clusters=3, init='k-means++', n_init=10, max_iter=300, random_state=42)
modelo.fit(Entrada)
labels = modelo.labels_

## Questão 4 - Fórmulas Matemáticas

**Distância Euclidiana:**  
\[ d(\vec{x}, \vec{y}) = \sqrt{\sum_{i=1}^n (x_i - y_i)^2} \]

**Silhouette Score:**  
\[ s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))} \]

- \(a(i)\): distância média entre o ponto e os pontos do mesmo cluster.
- \(b(i)\): distância média até o cluster mais próximo.

## Questão 5 - Métrica adicional: Davies-Bouldin

In [ ]:
score_db = davies_bouldin_score(Entrada, labels)
print("Davies-Bouldin Index:", score_db)

## Questão 6 - DBSCAN e SOM

In [ ]:
# DBSCAN
modelo_db = DBSCAN(eps=0.3, min_samples=5)
labels_db = modelo_db.fit_predict(Entrada)

# SOM
som = MiniSom(x=3, y=1, input_len=4, sigma=1.0, learning_rate=0.5)
som.random_weights_init(Entrada)
som.train_random(Entrada, 100)
labels_som = np.array([som.winner(x)[0] for x in Entrada])

print("KMeans clusters:", len(set(labels)))
print("DBSCAN clusters:", len(set(labels_db)) - (1 if -1 in labels_db else 0))
print("SOM clusters:", len(set(labels_som)))

## Questão 7 - Instâncias agrupadas incorretamente

In [ ]:
labels_reais = base['Species'][:len(Entrada)]
Entrada_2D = PCA(n_components=2).fit_transform(Entrada)
df_vis = pd.DataFrame(Entrada_2D, columns=['PC1', 'PC2'])
df_vis['Cluster'] = labels
df_vis['Classe'] = labels_reais.values

sns.scatterplot(data=df_vis, x='PC1', y='PC2', hue='Classe', style='Cluster')
plt.title("Comparação entre Classe e Cluster")
plt.show()

print("ARI:", adjusted_rand_score(labels_reais, labels))

## Questão 8 - Relatório Final

- **Base utilizada**: Iris
- **Pré-processamento**: Normalização + remoção de outliers
- **Algoritmos**: KMeans, DBSCAN, SOM
- **Avaliação**: Elbow, Silhouette, Davies-Bouldin, ARI
- **Conclusão**: KMeans se adequa bem à base Iris, mas DBSCAN pode encontrar formas mais complexas.